У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
from imblearn.over_sampling import SMOTENC
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from imblearn.combine import SMOTETomek
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import classification_report

import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("drive/MyDrive/ML_Course/Data/customer_segmentation/customer_segmentation_train.csv", index_col=0)

In [4]:
df.head(5)

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
ID,,,,,,,,,,
462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [5]:
df.shape

(8068, 10)

Датасет має 8068 екземплярів та 10 колоночек.

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8068 entries, 462809 to 461879
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           8068 non-null   object 
 1   Ever_Married     7928 non-null   object 
 2   Age              8068 non-null   int64  
 3   Graduated        7990 non-null   object 
 4   Profession       7944 non-null   object 
 5   Work_Experience  7239 non-null   float64
 6   Spending_Score   8068 non-null   object 
 7   Family_Size      7733 non-null   float64
 8   Var_1            7992 non-null   object 
 9   Segmentation     8068 non-null   object 
dtypes: float64(2), int64(1), object(7)
memory usage: 693.3+ KB


Відповідно до отриманого звіту, маємо пропущені дані в датасеті. Для обрання стратегії заповнення пропущених даних кожної змінної, необхідно детальніше дослідити інформацію про розподіл даних в цих змінних.

In [7]:
null_count = df.isnull().sum()
null_percentage = round((df.isnull().sum()/df.shape[0])*100, 4)
null_df = pd.DataFrame({'column_name' : df.columns,'null_count' : null_count,'null_percentage': null_percentage})
null_df.reset_index(drop = True, inplace = True)
null_df.sort_values(by = 'null_percentage', ascending = False)

,column_name,null_count,null_percentage
5,Work_Experience,829,10.2752
7,Family_Size,335,4.1522
1,Ever_Married,140,1.7353
4,Profession,124,1.5369
3,Graduated,78,0.9668
8,Var_1,76,0.9420
0,Gender,0,0.0000
2,Age,0,0.0000
6,Spending_Score,0,0.0000
9,Segmentation,0,0.0000


Найбільшу кількість пропущених значень (10%) має колоночка Work_Experience, проте цей відсоток не є критичним, тому приймаємо рішення заповнити пропущенні дані, а не видаляти. Особливо зогляду на те, що датасет містить не надто багато екземплярів і видалення призведе до втрати інформації.  
По іншим колоночкам відсоток відсутніх значень є ще меншим, тому по ним так само будуть заповнені значення

In [8]:
numeric_columns = df.select_dtypes('number').columns
categorical_columns = df.select_dtypes('object').columns
numeric_columns, categorical_columns

(Index(['Age', 'Work_Experience', 'Family_Size'], dtype='object'),
 Index(['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score',
        'Var_1', 'Segmentation'],
       dtype='object'))

Датасет містить 3 змінних з числовим типом даних та 6 змінних категоріальних. Таргет змінна Segmentation такоє є категоріальною.

In [9]:
df.describe()

,Age,Work_Experience,Family_Size
count,8068.000000,7239.000000,7733.000000
mean,43.466906,2.641663,2.850123
std,16.711696,3.406763,1.531413
min,18.000000,0.000000,1.000000
25%,30.000000,0.000000,2.000000
50%,40.000000,1.000000,3.000000
75%,53.000000,4.000000,4.000000
max,89.000000,14.000000,9.000000


Age: медіана і середнє близькі по значенню, проте велика різниця між 75 персентилем і макисмальним значенням може вказувати на наявність викидів.  
Work_Experience: стандартне відхилення більше за середнє, що вказує на наявність викидів так само, як і велика різниця між 75 персентилем і максимальним значенням. Медіана значно менша за середнє значення.  
Family_Size: медіана і середнє близькі за значенням. Можлива наявність викидів.  
**Висновок:**  
В якості майбутньої стратегії заповнення пропущених значень обираємо заповнення медіанним значенням для всіх числових змінних.

In [10]:
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline

In [11]:
for col in numeric_columns:

  fig = px.histogram(df,
                    x=col,
                    marginal='box',
                    nbins=50,
                    title=f'Distribution of {col}')
  fig.update_layout(bargap=0.1)
  fig.show()

Розподіли числових змінних візуально не є нормальними. Дійсно присутні викиди. Тому в якості стратегії масштабування даних обираємо RobustScaler оскільки він є більш стійким до викидів.

In [12]:
for col in categorical_columns:
  print(df[col].value_counts())
  print()

Gender
Male      4417
Female    3651
Name: count, dtype: int64

Ever_Married
Yes    4643
No     3285
Name: count, dtype: int64

Graduated
Yes    4968
No     3022
Name: count, dtype: int64

Profession
Artist           2516
Healthcare       1332
Entertainment     949
Engineer          699
Doctor            688
Lawyer            623
Executive         599
Marketing         292
Homemaker         246
Name: count, dtype: int64

Spending_Score
Low        4878
Average    1974
High       1216
Name: count, dtype: int64

Var_1
Cat_6    5238
Cat_4    1089
Cat_3     822
Cat_2     422
Cat_7     203
Cat_1     133
Cat_5      85
Name: count, dtype: int64

Segmentation
D    2268
A    1972
C    1970
B    1858
Name: count, dtype: int64



Gender: кількість чоловіків в представленому датасеті переважає кількість жінок. Перевага не значна, проте присутня. Дана змінна не містить пропущених значень у вхідному датасеті, проте для можливих пропущених даних в проде, я би обрала стратегію заповнення пропущених значень - strategy='most_frequent'.  
Ever_Married: це також бінарна категоріальна змінна. Переважають значення тих, хто був хоч раз одружений. Для заповнення пропущених значень так само обираємо strategy='most_frequent'.  
Graduated: аналогічну стратегію обираємо і для цієї змінної.  
Profession: найбільш численими категоріями в цій змінній є Artist, Healthcare та Entertainment. Проте кращим варіантом для заповнення пропущених значень є введення нової категорії Other. Це так само буде корисним, якщо на тестових даних до нас потрапить невідома нова категорія.  
Spending_Score: дана змінна містить 3 категорії, які мають логічний порядок. Найбільше представлена категорія Low. Хоча дана змінна не містить пропущених значень, для можливих пропущених даних в проде, я би обрала стратегію заповнення пропущених значень найчастотним значенням.  
Var_1: змінна містить 6 різних категорій, найчастотніша з яких 6 категорія з великим відривом від інших. Для збереження розподілу, заповнимо пропущенні значення саме цим значенням.

In [13]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['Segmentation'])
input_cols = list(train_df.columns)[:-1]
target_col = 'Segmentation'
train_inputs = train_df[input_cols]
train_targets = train_df[target_col]
val_inputs = val_df[input_cols]
val_targets = val_df[target_col]

In [14]:
numeric_cols = train_inputs.select_dtypes('number').columns.tolist()
categorical_cols = train_inputs.select_dtypes('object').columns.tolist()

In [15]:
label_cols = train_inputs[categorical_cols].drop(columns=['Profession', 'Spending_Score', 'Var_1']).columns.tolist()
ordered_cols = 'Spending_Score'
onehot_cols = train_inputs[categorical_cols].drop(columns=['Gender', 'Ever_Married', 'Graduated', 'Spending_Score']).columns.tolist()

In [16]:
imputer_num = SimpleImputer(strategy='median')
imputer_cat = SimpleImputer(strategy='most_frequent')

imputer_num.fit(train_inputs[numeric_cols])
imputer_cat.fit(train_inputs[categorical_cols])

SimpleImputer(strategy='most_frequent')

In [17]:
train_inputs[numeric_cols] = imputer_num.transform(train_inputs[numeric_cols])
train_inputs[categorical_cols] = imputer_cat.transform(train_inputs[categorical_cols])

val_inputs[numeric_cols] = imputer_num.transform(val_inputs[numeric_cols])
val_inputs[categorical_cols] = imputer_cat.transform(val_inputs[categorical_cols])

In [18]:
train_inputs.info(), val_inputs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6454 entries, 465905 to 459407
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           6454 non-null   object 
 1   Ever_Married     6454 non-null   object 
 2   Age              6454 non-null   float64
 3   Graduated        6454 non-null   object 
 4   Profession       6454 non-null   object 
 5   Work_Experience  6454 non-null   float64
 6   Spending_Score   6454 non-null   object 
 7   Family_Size      6454 non-null   float64
 8   Var_1            6454 non-null   object 
dtypes: float64(3), object(6)
memory usage: 504.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 462588 to 461114
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           1614 non-null   object 
 1   Ever_Married     1614 non-null   object 
 2   Age              1614 non-null   fl

(None, None)

In [19]:
scaler = RobustScaler()
scaler.fit(train_inputs[numeric_cols])

train_inputs[numeric_cols] = scaler.transform(train_inputs[numeric_cols])
val_inputs[numeric_cols] = scaler.transform(val_inputs[numeric_cols])

In [20]:
label_encoder = LabelEncoder()

for col in label_cols:
  label_encoder.fit(train_inputs[col])
  train_inputs[f'encoded_{col}'] = label_encoder.transform(train_inputs[col])
  val_inputs[f'encoded_{col}'] = label_encoder.transform(val_inputs[col])

In [21]:
categories = [train_inputs[ordered_cols].unique().tolist()]
ordinal_encoder = OrdinalEncoder(categories=categories)

ordinal_encoder.fit(train_inputs[[ordered_cols]])
train_inputs['encoded_Spending_score'] = ordinal_encoder.transform(train_inputs[[ordered_cols]])
val_inputs['encoded_Spending_score'] = ordinal_encoder.transform(val_inputs[[ordered_cols]])

In [22]:
onehot_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

onehot_encoder.fit(train_inputs[onehot_cols])
onehot_encoded_cols = list(onehot_encoder.get_feature_names_out(onehot_cols))
train_inputs[onehot_encoded_cols] = onehot_encoder.transform(train_inputs[onehot_cols])
val_inputs[onehot_encoded_cols] = onehot_encoder.transform(val_inputs[onehot_cols])



In [23]:
X_train = train_inputs.drop(columns=categorical_cols)
X_val = val_inputs.drop(columns=categorical_cols)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [24]:
all_columns = X_train.columns.tolist()
categorical_indices = [all_columns.index(col) for col in all_columns if col not in numeric_columns]

smote_nc = SMOTENC(categorical_features=categorical_indices, random_state=42, k_neighbors=10)
X_resampled_smote_nc, y_resampled_smote_nc = smote_nc.fit_resample(X_train, train_targets)

In [25]:
smote = SMOTE(k_neighbors=10, random_state=42)
tomek = TomekLinks(sampling_strategy='all')

smote_tomek = SMOTETomek(smote=smote, tomek=tomek, random_state=42)
X_resampled_smote_tomek, y_resampled_smote_tomek = smote_tomek.fit_resample(X_train, train_targets)

**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [26]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train, train_targets)
ovr_predictions = ovr_model.predict(X_val)

print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.41      0.45      0.43       394
           B       0.43      0.15      0.23       372
           C       0.49      0.65      0.56       394
           D       0.64      0.75      0.69       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.50      0.51      0.49      1614



In [27]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_resampled_smote_nc, y_resampled_smote_nc)
ovr_predictions = ovr_model.predict(X_val)

print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.42      0.47      0.44       394
           B       0.43      0.22      0.29       372
           C       0.50      0.63      0.56       394
           D       0.67      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614



In [28]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_resampled_smote_tomek, y_resampled_smote_tomek)
ovr_predictions = ovr_model.predict(X_val)

print(classification_report(val_targets, ovr_predictions))

              precision    recall  f1-score   support

           A       0.43      0.49      0.46       394
           B       0.44      0.21      0.28       372
           C       0.50      0.64      0.56       394
           D       0.67      0.71      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.52      0.52      0.51      1614



Для оцінки якості моделі я обираю показник macro F1-score, тому що відповідно до задачі, наша модель має однаково передбачати всі 4 групи клієнтів. У нас не задано, що якийсь клас має переважне значення над іншими. А також, немає значення кількість екземплярів в кожному класі.  
Як ми бачимо, всі три моделі мають дуже схожий показник macro F1-score і тільки модель, натренована на оригінальному датасеті має трохи менше значення.  
Припускаю, що причиною цьому є відсутність явного дисбалансу класів в нашому датасеті, через що методи ресемплінгу не дали сильного покращення моделі.  
Крім того, можу відмітити, що в усіх моделях дуже низький показник recall для класу В, що означає, що модель має труднощі з визначенням саме цього класу.  Можливо, це через те, що клас В схожий на інші класи. З цього я можу зробити висновок, що для покращення моделі слід сфокусуватись на частині фіча інжинірингу, щоб в цілому збагатити дані, а в ідеальному випадку, створити нові фічі, на основі яких модель зможе покращити передбачення як класу В, так і інших класів.  
Мені вдалось покращити значення recall для клсу В в моделі з SMOTENC, використавши tomek = TomekLinks(sampling_strategy='all') замість tomek = TomekLinks(sampling_strategy='auto'), тобто зберігши баланс класів.
Відповідаючи на питання, яка з моделей найкраща, я би сказала модель зі SMOTENC